## 대학원 입학 예측(회귀)
- 예측할 값(target): "Chance of Admit "
- 평가: r2
- data(3개): t2-2-X_train, t2-2-y_train, t2-2-X_test
- 제출 형식(Serial No.-> id, 예측 값 -> target)

~~~
id,target
28,0.741696
76,0.779616
151,0.897247
~~~

## Baseline
- 데이터 크기가 작아 크로스 밸리데이션만 확인했어요 
- 타겟 피처에 공백이 있어요 "Chance of Admit "
- r2로 평가해요

확률을 예측(회귀 문제)
불러오기 - eda -전처리 - 모델링(랜덤포레스트리그레서 사용예정) -평가 (score, predict_proba)

In [1]:
# 라이브러리 불러오기
import pandas as pd

In [2]:
# 데이터 불러오기
X_train = pd.read_csv("../input/big-data-analytics-certification/t2-2-X_train.csv")
y_train = pd.read_csv("../input/big-data-analytics-certification/t2-2-y_train.csv")
X_test = pd.read_csv("../input/big-data-analytics-certification/t2-2-X_test.csv")

X_train.shape, y_train.shape, X_test.shape

((320, 8), (320, 2), (80, 8))

ID -> CHANCE OF ADMIT 예측 (회귀) - EDA - 전처리 =모델링 -평가

## EDA

In [3]:
X_train.shape, X_test.shape, y_train.shape #행(가로,rows,axis=0),열(세로,columns,axis=1)확인

((320, 8), (80, 8), (320, 2))

In [4]:
X_train.head()
X_train.info() #문자형 없음
X_train.isnull().sum()
X_test.isnull().sum() #결측치 없음
X_train.describe() #스케일링 필요
y_train.describe() #따로 할필요 없음


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 320 entries, 0 to 319
Data columns (total 8 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Serial No.         320 non-null    int64  
 1   GRE Score          320 non-null    int64  
 2   TOEFL Score        320 non-null    int64  
 3   University Rating  320 non-null    int64  
 4   SOP                320 non-null    float64
 5   LOR                320 non-null    float64
 6   CGPA               320 non-null    float64
 7   Research           320 non-null    int64  
dtypes: float64(3), int64(5)
memory usage: 20.1 KB


,Serial No.,Chance of Admit
count,320.000000,320.000000
mean,199.793750,0.720281
std,115.308288,0.140227
min,1.000000,0.340000
25%,98.750000,0.630000
50%,201.500000,0.725000
75%,299.250000,0.820000
max,400.000000,0.970000


eda = id 뺴내고, 스케일링

In [5]:
X_train_id = X_train.pop('Serial No.')
X_test_id = X_test.pop('Serial No.')

In [6]:
import xgboost
from xgboost import XGBClassifier
print(help(XGBClassifier))

Help on class XGBClassifier in module xgboost.sklearn:

class XGBClassifier(XGBModel, sklearn.base.ClassifierMixin)
 |  XGBClassifier(*, objective: Union[str, Callable[[numpy.ndarray, numpy.ndarray], Tuple[numpy.ndarray, numpy.ndarray]], NoneType] = 'binary:logistic', use_label_encoder: bool = False, **kwargs: Any) -> None
 |  
 |  Implementation of the scikit-learn API for XGBoost classification.
 |  
 |  
 |  Parameters
 |  ----------
 |  
 |      n_estimators : int
 |          Number of boosting rounds.
 |  
 |      max_depth :  Optional[int]
 |          Maximum tree depth for base learners.
 |      max_leaves :
 |          Maximum number of leaves; 0 indicates no limit.
 |      max_bin :
 |          If using histogram-based algorithm, maximum number of bins per feature
 |      grow_policy :
 |          Tree growing policy. 0: favor splitting at nodes closest to the node, i.e. grow
 |          depth-wise. 1: favor splitting at nodes with highest loss change.
 |      learning_rate : 

In [7]:
import sklearn.metrics
from sklearn.metrics import f1_score
print(help(sklearn.metrics.f1_score))

Help on function f1_score in module sklearn.metrics._classification:

f1_score(y_true, y_pred, *, labels=None, pos_label=1, average='binary', sample_weight=None, zero_division='warn')
    Compute the F1 score, also known as balanced F-score or F-measure.
    
    The F1 score can be interpreted as a harmonic mean of the precision and
    recall, where an F1 score reaches its best value at 1 and worst score at 0.
    The relative contribution of precision and recall to the F1 score are
    equal. The formula for the F1 score is::
    
        F1 = 2 * (precision * recall) / (precision + recall)
    
    In the multi-class and multi-label case, this is the average of
    the F1 score of each class with weighting depending on the ``average``
    parameter.
    
    Read more in the :ref:`User Guide <precision_recall_f_measure_metrics>`.
    
    Parameters
    ----------
    y_true : 1d array-like, or label indicator array / sparse matrix
        Ground truth (correct) target values.
    

## 전처리

In [8]:
from sklearn.preprocessing import MinMaxScaler
model = MinMaxScaler()
X_train = model.fit_transform(X_train)
X_test = model.transform(X_test)

In [9]:
target = y_train["Chance of Admit "]


In [10]:
import sklearn.model_selection
from sklearn.model_selection import cross_val_score
print(help(sklearn.model_selection.cross_val_score)) #cross_val_score(--, x, y, scoring='r2', cv=5)

Help on function cross_val_score in module sklearn.model_selection._validation:

cross_val_score(estimator, X, y=None, *, groups=None, scoring=None, cv=None, n_jobs=None, verbose=0, fit_params=None, pre_dispatch='2*n_jobs', error_score=nan)
    Evaluate a score by cross-validation.
    
    Read more in the :ref:`User Guide <cross_validation>`.
    
    Parameters
    ----------
    estimator : estimator object implementing 'fit'
        The object to use to fit the data.
    
    X : array-like of shape (n_samples, n_features)
        The data to fit. Can be for example a list, or an array.
    
    y : array-like of shape (n_samples,) or (n_samples, n_outputs),             default=None
        The target variable to try to predict in the case of
        supervised learning.
    
    groups : array-like of shape (n_samples,), default=None
        Group labels for the samples used while splitting the dataset into
        train/test set. Only used in conjunction with a "Group" :term:`cv

In [11]:
from sklearn.ensemble import RandomForestRegressor
rf = RandomForestRegressor(n_estimators = 100, random_state=2021)
rf.fit(X_train, target)
rf_score = cross_val_score(rf, X_train, target, scoring='r2', cv=5)
rf_score.mean()

0.7468740308446437

In [12]:
import sklearn.tree
print(sklearn.tree.__all__)
from sklearn.tree import DecisionTreeRegressor
# print(help(sklearn.tree.DecisionTreeRegressor)) #max_depth

['BaseDecisionTree', 'DecisionTreeClassifier', 'DecisionTreeRegressor', 'ExtraTreeClassifier', 'ExtraTreeRegressor', 'export_graphviz', 'plot_tree', 'export_text']


In [13]:
dt = DecisionTreeRegressor(max_depth=4)
dt.fit(X_train, target)
dt_score = cross_val_score(dt, X_train, target, scoring='r2', cv=5)
dt_score.mean()

0.7239965372890353

In [14]:
import sklearn
print(sklearn.__all__)

['calibration', 'cluster', 'covariance', 'cross_decomposition', 'datasets', 'decomposition', 'dummy', 'ensemble', 'exceptions', 'experimental', 'externals', 'feature_extraction', 'feature_selection', 'gaussian_process', 'inspection', 'isotonic', 'kernel_approximation', 'kernel_ridge', 'linear_model', 'manifold', 'metrics', 'mixture', 'model_selection', 'multiclass', 'multioutput', 'naive_bayes', 'neighbors', 'neural_network', 'pipeline', 'preprocessing', 'random_projection', 'semi_supervised', 'svm', 'tree', 'discriminant_analysis', 'impute', 'compose', 'clone', 'get_config', 'set_config', 'config_context', 'show_versions']


In [15]:
rf.fit(X_train, target)
pred = rf.predict(X_test)
pred

array([0.7154, 0.7045, 0.9165, 0.6796, 0.5074, 0.7906, 0.4699, 0.4696,
       0.5608, 0.7772, 0.6547, 0.8564, 0.5254, 0.6994, 0.4633, 0.7599,
       0.9265, 0.9414, 0.9676, 0.6388, 0.6961, 0.7234, 0.7453, 0.8222,
       0.7125, 0.6785, 0.9208, 0.6538, 0.9608, 0.6704, 0.7343, 0.4756,
       0.6957, 0.7488, 0.669 , 0.9463, 0.6127, 0.6518, 0.9337, 0.7297,
       0.6425, 0.7642, 0.6854, 0.677 , 0.8312, 0.6841, 0.5504, 0.7278,
       0.8055, 0.8412, 0.5363, 0.8697, 0.9356, 0.738 , 0.7723, 0.6529,
       0.5825, 0.5937, 0.7935, 0.8984, 0.8135, 0.6751, 0.731 , 0.7205,
       0.624 , 0.699 , 0.6965, 0.9308, 0.7104, 0.8949, 0.6169, 0.69  ,
       0.7427, 0.9122, 0.9146, 0.7397, 0.9301, 0.9329, 0.8223, 0.7381])

In [16]:
output =pd.DataFrame({'id' : X_test_id, 'target' : pred})

In [17]:
output.head()
output.to_csv("220623.csv", index=False)
output

,id,target
0,128,0.7154
1,76,0.7045
2,151,0.9165
3,102,0.6796
4,272,0.5074
...,...,...
75,252,0.7397
76,362,0.9301
77,338,0.9329
78,186,0.8223


## 모델 학습 및 평가

- 밸리데이션 데이터를 나누기엔 데이터 자체가 너무 작아 크로스밸리데이션만 확인함

## csv 확인

In [19]:
pd.read_csv("220623.csv")

,id,target
0,128,0.7154
1,76,0.7045
2,151,0.9165
3,102,0.6796
4,272,0.5074
...,...,...
75,252,0.7397
76,362,0.9301
77,338,0.9329
78,186,0.8223
